In [60]:
import requests
import json
import pandas as pd
import csv


# Set the API endpoint and parameters
endpoint = "https://api.census.gov/data/2019/acs/acs5"
params = {
    "get": "NAME,B25040_001E,B25040_002E,B25040_003E,B25040_004E,B25040_005E,B25040_006E,B25040_007E,B25040_008E,B25040_009E,B25040_010E",
    "for": "county:*",
    "in": "state:08",
    "key": "8588b7d546ae7ff3ee03bc7649d83defea3fb121"
}

# Make the API request
response = requests.get(endpoint, params=params)

# Check if the request was successful
if response.status_code == 200:
    # Parse the response as JSON
    data = response.json()

    # Assign the correct headers
    headers = data[0]
    headers.append("state")  # add "state" to headers
    data = data[1:]

    # Get the variable labels from the API documentation
    var_def_endpoint = "https://api.census.gov/data/2019/acs/acs5/variables.json"
    var_def_response = requests.get(var_def_endpoint)
    var_def_data = var_def_response.json()
    
    labels = []
    for variable_code in headers:
        if variable_code == "NAME":
            variable_label = "Name"
        elif variable_code == "state":
            variable_label = "State"
        elif variable_code == "county":
            variable_label = "County"
        else:
            variable_label = var_def_data["variables"][variable_code]["label"]
        labels.append(variable_label)
        
        # Print the headers with labels
    print(labels)
    print(headers)
    
    # Print the first 300 rows of data with correct headers
    for row in data[:300]:
        print(row)

    # Write the data to a CSV file
    with open("census_data.csv", "w", newline="") as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(labels)
        writer.writerows(data)

    print("Data successfully exported to census_data.csv.")
else:
    print(f"API request failed with status code {response.status_code}.")